# Tapestri analysis using the (customized) Mosaic package

### - **1. load and genotype SNV matrix**
### - **plot single-cell mutational-prevalence histogram**
### - **plot single-cell heatmap for SNV data**

Notes:
1. Official tutorial by Mission Bio on the data structure and basic usage of Mosaic (v1.8) can be found at 

    - Data structure: https://missionbio.github.io/mosaic/v1.8.0/index.html

    - Usage Vignettes: https://github.com/MissionBio/mosaic-jupyter

2. The latest version of Mosaic by Mission Bio is v2.0 (as of 2022-06-02). This customized Mosaic package is based on Mosaic v1.6.0, so some new features mentioned in the official tutorial (v1.8) might not be available in this customized version.  

3. This notebook mainly aims to demonstrate the customized functionalities on top of Mission Bio's Mosaic package (v1.6.0) for Iacobuzio lab's PDAC sample analysis as described in the manuscript Zhang et al.(2022).

## -- (0). import packages; set up input files & directories.

In [1]:
import mosaic.io as mio
from pathlib import Path
import pandas as pd

# plotting
import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio
config = {
  'toImageButtonOptions': {
    'format': 'png', # one of png, svg, jpeg, webp
    'filename': 'MyImage',
    'scale': 3 # Multiply title/legend/axis/canvas sizes by this factor
  }
}
pio.renderers.default = "notebook"

<frozen importlib._bootstrap>:228: RuntimeWarning: scipy._lib.messagestream.MessageStream size changed, may indicate binary incompatibility. Expected 56 from C header, got 64 from PyObject


In [5]:
sample_names = ['[example]-organoid']

data_dir = Path.cwd() / '..' / 'example' 
wd = data_dir
print(data_dir)

/Users/haochen/Desktop/software/mosaic/tutorials/../example


## -- (1) load and set up Tapestri "Sample" object

After loading the H5 file into a mosaic.Sample object, we will **genotype** the `sample.dna` layer. 

The function `sample.dna.genotype()` can take inputs:
- het_vaf [default: 20]: minimum variant allele frequency (VAF) to assign an SNV in a cell as heterozygous (`HET`, numbered `1`).
- hom_vaf [default: 80]: minimum variant allele frequency (VAF) to assign an SNV in a cell as homozygous (`HOM`, numbered `2`).
- min_dp [default: None]: minimum depth to assign an SNV in a cell as missing (`MISSING`, numbered `3`).
- min_alt_read [default: None]: if an SNV with a HET/HOM genotype has below (strictly) this number of reads in a cell, it will be assigned as wildtype (`WT`, numbered `0`).
- min_gq [default: 0]): if an SNV with a HET/HOM genotype has below (strictly) this genotype quality, it will be assigned as wildtype (`WT`, numbered `0`). 

Subsequently, the function will add an `NGT` and a `mut` layer to `sample.dna`. The `NGT` layer assigns the corresponding number to each SNV in each cell given its genotype; the `mut` layer is a binary layer indicating whether an SNV is present (mutated, with genotype `HET` or `HOM`) or not (`WT` or `MISSING`) in each cell.

In [6]:
sample_objs = {}

for sample_i in sample_names:
    input_h5 = data_dir / f"{sample_i}_DNA_CNV.h5"
    # 1 ----- read h5 into mosaic.assay object -----
    sample_obj = mio.load(str(input_h5), name = sample_i, raw=False)

    # 2 ----- standard genotyping -----
    sample_obj.dna.genotype_variants(min_dp = 5,
                                min_alt_read = 3)

    sample_objs[sample_i] = sample_obj

Loading, /Users/haochen/Desktop/software/mosaic/tutorials/../example/[example]-organoid_DNA_CNV.h5
Loaded in 0.5s.


We can call the function `sample.dna.shape` to inspect the shape of the SNV by single-cell matrix:

In [10]:
sample_objs['organoid_example'].dna.shape

(1076, 884)

In [17]:
sample_objs['organoid_example'].dna.get_attribute(
    'mut', features = ['chr12:25398284:C/T']
).sum()[0]

1069

## -- (2). plot mutational prevalence

Now that we have genotyped every SNV in each single cell and added the 'mut' layer, we can use it to calculate the single-cell mutational prevalence for each SNV (in how many single cells each SNV is mutated) and plot the distribution.

For this we use the `tea.plots.plot_var_sc_mutational_prev()` function, which takes as input:

- `sample_obj`: a mosaic.Sample object. The `mut` layer must be present.
- `min_mut_prev_of_interest`: float [default: 0.01]. The mutational prevalence to highlight on the histogram.
- `vars_to_highlight`: a dictionary with keys as SNV names and values as the corresponding SNV annotation.
- `sample_name`: string [default: `sample_obj.dna.metadata['sample_name']`]. The name of the sample to be used in the title.

### read in bulk  data

Because the novel variant discovery mechanism of the default Tapestri pipeline is yet to be validated, we will use matched bulk sequencing as a reference to obtain a list of high-confidence SNVs for analysis.

The bulk-miseq (targeted sequencing) MAF (mutation annotation format) file of this patient has been processed with `bedtools` to narrow down to mutations that are covered by the custom PDAC panel (named `1440`).


In [42]:
bulk_tumor_maf = '/Users/haochen/Desktop/Tapestri_analysis_tutorial/data/organoid_example_miseq_on_panel1440.txt'
bulk_tumor_df = pd.read_csv(bulk_tumor_maf, sep='\t')
if not bulk_tumor_df['Chromosome'].astype(str).str.startswith('chr').any():
    bulk_tumor_df['Chromosome'] = 'chr' + bulk_tumor_df['Chromosome'].astype(str)
bulk_tumor_df['condensed_format'] = bulk_tumor_df['Chromosome'].astype(str) + ':' + bulk_tumor_df['Position'].astype(str) + ':' + bulk_tumor_df['Reference Allele'].astype(str) + '/' + bulk_tumor_df['Variant Allele'].astype(str)
bulk_tumor_df.set_index('condensed_format', inplace=True)

unique_bulk_vars = bulk_tumor_df.index.unique().tolist()
len(unique_bulk_vars)

11

We will put together a simple annotation map based on the bulk MAF

In [46]:
bulk_vars_ann_map = {}
for var_i in unique_bulk_vars:
    bulk_vars_ann_map[var_i] = bulk_tumor_df.loc[var_i, 'Gene ID']
bulk_vars_ann_map

{'chr3:52637486:C/G': 'PBRM1',
 'chr3:52676064:AC/A': 'PBRM1',
 'chr3:52676065:C/T': 'PBRM1',
 'chr3:52676066:A/T': 'PBRM1',
 'chr10:89720633:CT/C': 'PTEN',
 'chr11:64572557:A/G': 'MAP4K2,MEN1',
 'chr12:25398284:C/T': 'KRAS',
 'chr17:7578474:C/CG': 'TP53',
 'chr17:7579472:G/C': 'TP53',
 'chr17:7579643:CCCCCAGCCCTCCAGGT/C': 'TP53',
 'chr17:41245466:G/A': 'BRCA1'}

### Now, we can plot the mutational prevalence histogram for SNVs, and highlight the SNVs identified by bulk-WES

In [29]:
import tea.plots

In [47]:
mut_prev_histogram = tea.plots.plot_var_sc_mutational_prev(
    sample_obj = sample_objs['organoid_example'],
    min_mut_prev_of_interest = 0.005,
    vars_to_highlight = bulk_vars_ann_map,
    sample_name = 'organoid_example',
)

mut_prev_histogram.show(config=config)

***Note***: this is H5 file is a heavily filtered version of the original H5 file, and corresponds to a sample not present in the sample cohort presented in Zhang et al. (2022).


## -- (3). plot single-cell clonal relationship for variants of interest

Now, we want to see how SNVs of interest colocalize at the single-cell level. 

For this, we use the `tea.plots.plot_snv_clone()` function, which is a wrapper of the `mosaic.sample.dna.heatmap()` function. The function takes as input:

- `sample_obj`: a mosaic.Sample object.
- `sample_name`: string [default: `sample_obj.dna.metadata['sample_name']`]. 
  - The name of the sample to be used in the title.
- `story_topic`: string. 
  - The story topic to be used in the title.
- `voi`: a list of SNVs to be plotted. 
- `attribute`: string. 
  - The attribute layer of the `sample_obj.dna` to be plotted (e.g. `NGT` or `AF_MISSING`). Note the difference between layers `AF` and `AF_MISSING` is that the former has `MISSING` values as `0` while the latter has them as `-50`, and therefore the latter is better for plotting purpose.
- `vars_to_sort_by`: a list of SNVs to sort single cells by [default: the first SNV in `voi`].
- `barcode_sort_method`: `hier` or `single-var` [default: `hier`]. 
  - the method to sort single cells on the given SNV(s), either hierarchical clustering or naive ordering (see https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.sort_values.html).
- `ann_map`: a dictionary with keys as SNV names and values as the corresponding SNV annotation.

The function returns a highly customizable plotly.graph_object 

In [52]:
sample_of_interest = 'organoid_example'
story_topic = 'bulk_vars'
attribute = 'AF_MISSING'
voi = unique_bulk_vars
# make output directory 
output_dir = wd / f'{sample_of_interest}-sc_heatmaps-{story_topic}'
if not output_dir.exists():
    output_dir.mkdir()

sc_heatmap = tea.plots.plot_snv_clone(
    sample_obj = sample_objs[sample_of_interest], 
    sample_name = sample_of_interest, 
    story_topic = story_topic, 
    voi = voi, 
    attribute = attribute, 
    ann_map = bulk_vars_ann_map, 
    vars_to_sort_by = voi
    )

[WARNING] --- chr3:52676064:AC/A not in dna.ids
[WARNING] --- chr3:52676065:C/T not in dna.ids
[WARNING] --- chr3:52676066:A/T not in dna.ids
chr3:52676066:A/T not in ids
chr3:52676065:C/T not in ids
chr3:52676064:AC/A not in ids


In [53]:
sc_heatmap.show(config=config)

In [51]:
type(sc_heatmap)

plotly.graph_objs._figurewidget.FigureWidget